In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**1.데이터 살펴 보기**

In [ ]:
import pandas as pd

train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')
submission = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

print(train.head())
print(test.head())
print(submission.head())

train 데이터에만 Survived가 포함되어 있음

In [ ]:
train.info()

891건 중 Age, Cabin, Embarked은 누락된 정보가 존재

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def bar_chart(feature):
  survived = train.loc[train["Survived"]==1,feature].value_counts()  #해당 특징의 Survived 가 1인 갯수를 세어준다.
  dead = train.loc[train["Survived"]==0,feature].value_counts()
  data = pd.DataFrame([survived,dead],index = ["Survived","Dead"])
  data.plot(kind="bar",figsize=(15,10))
  plt.show()

#pclass에 따른 생존/사망자 확인
bar_chart("Pclass")

1등석은 생존자가, 3등석은 사망자가 더 많음.
지도학습에 가져가는 것이 좋음

In [ ]:
#fare 에 따른 생존/사망자 확인
facet = sns.FacetGrid(train,hue='Survived',aspect=4)
facet.map(sns.kdeplot,'Fare',shade=True)
facet.set(xlim=(0,train['Fare'].max()))
facet.add_legend()
plt.xlim(0,180)
plt.show()

Fare 즉 요금이 낮을 수록 사망자가 많음. 지도학습에 가져가는 것이 좋음

In [ ]:
#sex 에 따른 생존/사망자 확인
bar_chart('Sex')

남자보다 여자 생존자가 더 많음. 지도학습에 가져가는 것이 좋음.

In [ ]:
#age 에 따른 생존/사망자 확인
facet = sns.FacetGrid(train,hue='Survived',aspect=4)
facet.map(sns.kdeplot,'Age',shade=True)
facet.set(xlim=(0,train['Age'].max()))
facet.add_legend()
plt.xlim(0,180)
plt.show()

나이가 어릴 수록 생존률이 높음.

In [ ]:
train['Familysize']=train['SibSp'] + train['Parch']
#train.drop('SibSp',axis=1,inplace=True)
#train.drop('Parch',axis=1,inplace=True)

bar_chart('Familysize')

혼자탄 사람이 친인척과 같이 탄 사람보다 사망율이 더 높음. 지도학습에 가져가는 것이 좋음.

In [ ]:
#embarked 에 따른 생존/사망자
bar_chart('Embarked')

C 선착장에서 탄 고객이 생존률이 높음.

**2. 데이터 가공**

In [ ]:
#age 결측치 처리
#평균으로 결측치 채우기
train['Age'].fillna(train['Age'].mean(axis=0),inplace=True)

In [ ]:
# age 구간화
# 청소년 : 0
# 청년 : 1
# 장년 : 2
# 중년 : 3
# 노년 : 4
ranges=[0,16,26,36,62,100]
labels=[0,1,2,3,4]
train["AgeGroup"]=pd.cut(train['Age'],ranges,right=False,labels=labels)
train.head()

In [ ]:
# fare 구간화

ranges=[0,17,30,100,1000]
labels=[0,1,2,3]
train["FareGroup"]=pd.cut(train['Fare'],ranges,right=False,labels=labels)
train.head()

In [ ]:
#성별을 숫자로 매핑
train['SexNum']=train['Sex'].apply(lambda x:1 if (x=='female') else 0)
train

In [ ]:
#embarked을 숫자로 매핑
train['EmbarkedNum']=train['Embarked'].apply(lambda x:0 if (x=='S') else (1 if(x=='C') else 2))
train

데이터 가공 후 다음과 같은 특성(Pclass,Familysize,AgeGroup,FareGroup,SexNum,EmbarkedNum)을 독립변수로 하여 Survived 를 예측하는 머신러닝 모델을 만들면 됨.

In [ ]:
target = train['Survived']
data = train[['Pclass','SexNum','AgeGroup','FareGroup','EmbarkedNum','Familysize']]

**3. 모델 선택**

In [ ]:
from sklearn.model_selection import train_test_split
#훈련세트와 테스트 세트 분리
train_input,test_input,train_target,test_target = train_test_split(data,target,test_size=0.2,random_state=42)

#어떤 알고리즘이 가장 성능이 좋은지 테스트 해 보자.
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import numpy as np

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

#K_fold cross validation 준비, 10개 구역을 나눔.
k_fold = KFold(n_splits=10,shuffle=True,random_state=42)

#KNN
knn = KNeighborsClassifier(n_neighbors=5)
score_knn = cross_val_score(knn,train_input,train_target,cv=k_fold,n_jobs=1,scoring='accuracy')

#의사결정 트리
dt = DecisionTreeClassifier()
score_dt = cross_val_score(dt,train_input,train_target,cv=k_fold,n_jobs=1,scoring='accuracy')

#랜덤 포레스트
rf = RandomForestClassifier(n_estimators=10)
score_rf = cross_val_score(rf,train_input,train_target,cv=k_fold,n_jobs=1,scoring='accuracy')

#SVM
sv = svm.SVC(gamma='auto')
score_sv = cross_val_score(sv,train_input,train_target,cv=k_fold,n_jobs=1,scoring='accuracy')

#정확도 확인
print("KNN   :",round(np.mean(score_knn)*100,2))
print("DecisionTree :",round(np.mean(score_dt)*100,2))
print("RandomForest :",round(np.mean(score_rf)*100,2))
print("SVM   :",round(np.mean(score_sv)*100,2))

SVM의 성능이 가장 좋음

**4.모델 훈련**

In [ ]:
#SVM으로 모델을 훈련하자
sv.fit(data,target)

**5.테스트 데이터 예측**

In [ ]:
#test 데이터도 train과 동일하게 데이터가공
test['Familysize']=test['SibSp'] + test['Parch']
#age 결측치 처리
#평균으로 결측치 채우기
test['Age'].fillna(test['Age'].mean(axis=0),inplace=True)

# age 구간화
# 청소년 : 0
# 청년 : 1
# 장년 : 2
# 중년 : 3
# 노년 : 4
ranges=[0,16,26,36,62,100]
labels=[0,1,2,3,4]
test["AgeGroup"]=pd.cut(test['Age'],ranges,right=False,labels=labels)
                        
# fare 구간화
ranges=[0,17,30,100,1000]
labels=[0,1,2,3]
test["FareGroup"]=pd.cut(test['Fare'],ranges,right=False,labels=labels)

#성별을 숫자로 매핑
test['SexNum']=test['Sex'].apply(lambda x:1 if (x=='female') else 0)
                        
#embarked을 숫자로 매핑
test['EmbarkedNum']=test['Embarked'].apply(lambda x:0 if (x=='S') else (1 if(x=='C') else 2))
                        
test_data = test[['Pclass','SexNum','AgeGroup','FareGroup','EmbarkedNum','Familysize']]
print(test_data)

In [ ]:
pred = sv.predict(test_data)

6.훈련 결과 제출

In [ ]:
submission = pd.DataFrame({
"PassengerId":test['PassengerId'],
"Survived": pred
})
  
submission.to_csv('submission.csv',index=False)

In [ ]:
submissionfile = pd.read_csv('/kaggle/working/submission.csv')

print(submissionfile.head())